In [120]:
import numpy as np
import math

In [200]:
N = 4
T = 2
arr_rate = 10
ps = 0.9
pb = 0.1
lambda_s = int(arr_rate*ps)
lambda_b = int(arr_rate*pb)
mu_s = 5
mu_b = 2

In [214]:
d1 = int(N + ((N*N)/(2*T)) - (N/2) + (N/T) + 1)
d2 = int((T)*(N/T) + 1)
print(d1)
print(d2)

q = np.zeros((d1,d1))
print(q)

def access_q(n1,s1,b1,n2,s2,b2):
    global q
    q_dim = ((q.shape[0]-d1)/d2)
    if n1 > q_dim:
        #extra = int((n1 - (int(q.shape[0]/d1)-1))*d1)
        extra = d2
        
        cc = q.shape[1]
        q = np.append(q,np.zeros((extra,cc)), axis=0)
        cr = q.shape[0]
        q = np.append(q,np.zeros((cr,extra)), axis=1)
    elif n2 > q_dim:
        #extra = int((n2 - (int(q.shape[0]/d1)-1))*d1)
        extra = d2
        #print(extra)
        
        cc = q.shape[1]
        q = np.append(q,np.zeros((extra,cc)), axis=0)
        cr = q.shape[0]
        q = np.append(q,np.zeros((cr,extra)), axis=1)
    
    
    row = access_map[(s1,b1)]
    col = access_map[(s2,b2)]
    if (n1>0):
        row = d1 + ((n1-1)*d2) + access_map_v2[(s1,b1)]
    if (n2>0):
        col = d1 + ((n2-1)*d2) + access_map_v2[(s2,b2)]
    return((row,col))
    

access_map = dict()
i = 0
for s in range(N+1):
    for b in range(int(N/T)+1):
        if (s+(b*T)) <= N:
            access_map[(s,b)] = i
            i = i+1
            
access_map_v2 = dict()
i = 0
for s in range(N+1):
    for b in range(int(N/T)+1):
        if (s+(b*T)) > (N-T) and (s+(b*T)) <= N:
            access_map_v2[(s,b)] = i
            i = i+1
access_map_v2

9
5
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


{(0, 2): 0, (1, 1): 1, (2, 1): 2, (3, 0): 3, (4, 0): 4}

In [191]:
def init_transitions_arr():
    global q
    for n in range(10):
        for s in range(N+1):
            for b in range (N+1):
                if (s+(b*T)) == N:
                    r,c = access_q(n,s,b, n+1,s,b)
                    q[r,c] = arr_rate
                if (s+(b*T)) <= (N-T) and n == 0:
                    r,c = access_q(0,s,b, 0,s+1,b)
                    q[r,c] = lambda_s
                    r,c = access_q(0,s,b, 0,s,b+1)
                    q[r,c] = lambda_b
                elif (s+(b*T)) > (N-T) and (s+(b*T)) < N and n == 0:
                    r,c = access_q(0,s,b, 0,s+1,b)
                    q[r,c] = lambda_s
                    r,c = access_q(0,s,b, 1,s,b)
                    q[r,c] = lambda_b
                        
def init_transitions_srv():
    global q
    for n in range(10):
        for s in range(N+1):
            for b in range (N+1):
                F = N - (b*T) - s
                
                if n > 0 and (s+(b*T)) == N:
                    K = min(n,T)
                    if s > 0:
                        r,c = access_q(n,s,b, n-1,s,b)
                        q[r,c] = q[r,c] + (s*mu_s*ps)

                        r,c = access_q(n,s,b, n,s-1,b)
                        q[r,c] = q[r,c] + (s*mu_s*pb)
                    if b > 0:
                        r,c = access_q(n,s,b, n-1,s,b)
                        q[r,c] = q[r,c] + (b*mu_b*pb)
                        
                        for i in range(1,K):
                            r,c = access_q(n,s,b, n-i,s+i,b-1)
                            q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,i)*pb)
                        
                        r,c = access_q(n,s,b, n-K,s+K,b-1)
                        q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,K))
                
    
                elif n > 0 and F > 0 and F < (T-1):
                    K = min(F,n-1)
                    if s > 0:
                        r,c = access_q(n,s,b, n,s-1,b)
                        q[r,c] = q[r,c] + (s*mu_s)
                    if b > 0:
                        for i in range(0,K):
                            r,c = access_q(n,s,b, n-i-1,s+i,b)
                            q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,i)*pb)
                            
                        r,c = access_q(n,s,b, n-K-1,s+K,b)
                        q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,K))
                    
                elif n > 0 and F == (T-1):
                    K = min(F,n-1)
                    if s > 0:
                        r,c = access_q(n,s,b, n-1,s-1,b+1)
                        q[r,c] = q[r,c] + (s*mu_s)
                    if b > 0:
                        for i in range(0,K):
                            r,c = access_q(n,s,b, n-i-1,s+i,b)
                            q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,i)*pb)
                            
                        r,c = access_q(n,s,b, n-K-1,s+K,b)
                        q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,K))
                        
                elif n == 0 and (s+(b*T)) <= N:
                    if s > 0:
                        r,c = access_q(0,s,b, 0,s-1,b)
                        q[r,c] = q[r,c] + (s*mu_s)
                    if b > 0:
                        r,c = access_q(0,s,b, 0,s,b-1)
                        q[r,c] = q[r,c] + (b*mu_b)

def fill_diagonal():
    for i in range(q.shape[0]):
        q[i,i] = -(np.sum(q[i]))

In [215]:
init_transitions_arr()
init_transitions_srv()
fill_diagonal()

In [210]:
r,c = access_q(2,0,2,1,0,2)
q[r,c]

0.4

In [226]:
diag = d1 + (d2*T)

Bs = []
for i in range(1,T+1):
    block = q[diag:diag+d2, diag-(d2*i):diag-(d2*i)+d2]
    Bs.append(block)
    
L = q[diag:diag+d2, diag:diag+d2]
F = q[diag:diag+d2, diag+d2:diag+d2+d2]
Bs[1]

array([[0.  , 0.  , 3.24, 0.  , 0.  ],
       [0.  , 0.  , 1.8 , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.62],
       [0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ]])

In [308]:
L_inv = np.linalg.inv(L)
V = np.matmul(F,L_inv)
Ws = []
for B in Bs:
    W = np.matmul(B,L_inv)
    Ws.append(W)
Ws

Rs = [np.zeros((d2,d2))]
for i in range(100):
    R = -V
    j = 2
    for W in Ws:
        R = np.add(R,-(np.matmul(np.linalg.matrix_power(Rs[i],j),W)))
        j += 1
    Rs.append(R)
R = Rs[-1]

#
tl = q[0:d1,0:d1]
tr = q[0:d1,d1:d1+d2]
bl = q[d1:d1+d2,0:d1]
for i in range(T-1):
    b = q[d1+((i+1)*d2) : d1+((i+1)*d2)+d2 , 0:d1]
    bl = np.add(bl,np.matmul(np.linalg.matrix_power(R,i+1),b))
br = L
i = 1
for b in Bs:
    br = np.add(br,np.matmul(np.linalg.matrix_power(R,i),b))
br

#
bc = np.append(tl,tr,axis=1)
bcb = np.append(bl,br,axis=1)
bc = np.append(bc,bcb,axis=0)

#
a = np.transpose(bc)
b = np.zeros(d1+d2)
b[-1] = 1
pi = np.linalg.solve(a,b)
pi

#
ir = np.linalg.inv(np.subtract(np.identity(R.shape[0]),R))
r = np.matmul(pi[d1:],ir)
alfa = np.sum(pi[:d1]) + np.sum(r)
pi = pi/alfa
pis = [pi[:d1],pi[d1:]]
for i in range(2,20):
    pis.append(np.matmul(pis[i-1],R))
pis

[array([0.08295999, 0.03796973, 0.00874593, 0.15073208, 0.0675386 ,
        0.13792642, 0.05938492, 0.08554465, 0.04103039]),
 array([0.00815642, 0.01624214, 0.04259487, 0.00911786, 0.02287342]),
 array([0.00724505, 0.0047665 , 0.0304248 , 0.00208508, 0.01366579]),
 array([0.00623852, 0.00347422, 0.02191462, 0.00134252, 0.00863376]),
 array([0.00525987, 0.00255698, 0.01594565, 0.00090633, 0.00574187]),
 array([0.00436943, 0.00190104, 0.01172908, 0.00063643, 0.00398834]),
 array([0.00359095, 0.00142737, 0.00872125, 0.00046097, 0.00286768]),
 array([0.00292787, 0.00108157, 0.00655143, 0.00034183, 0.00211664]),
 array([0.00237311, 0.00082629, 0.00496753, 0.00025794, 0.00159278]),
 array([0.00191485, 0.00063578, 0.00379772, 0.00019716, 0.00121556]),
 array([0.00153981, 0.00049218, 0.0029241 , 0.00015214, 0.0009372 ]),
 array([0.00123498, 0.00038296, 0.00226503, 0.00011823, 0.00072799]),
 array([9.88486595e-04, 2.99233202e-04, 1.76332494e-03, 9.23534523e-05,
        5.68593009e-04]),
 array

In [216]:
np.savetxt("q.csv", q,
              delimiter = ",")

In [309]:
sum([np.sum(pi) for pi in pis])

0.9972401525546938